From https://gitlab.com/dword4/nhlapi/tree/master

Import numpy, requests, and json modules

In [2]:
import numpy as np
#import pandas as pd
import requests
import json

Get the rosters for the 2017-2018 season as a json object

In [ ]:
team_rosters_2017_2018 = requests.get('https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster&season=20172018')

In [ ]:
team_rosters_2017_2018 = team_rosters_2017_2018.json()

Print out all the team names

In [ ]:
for i in range(0, len(team_rosters_2017_2018['teams'])):
    print(team_rosters_2017_2018['teams'][i]['name'])

Get every player from every team
Makes a players array holding [player_id, player_name, team]

In [ ]:
players_2017_2018 = []
for i in range(0, len(team_rosters_2017_2018['teams'])):
    #print(team_rosters_2017_2018['teams'][i]['name'])
    for j in range(0, len(team_rosters_2017_2018['teams'][i]['roster']['roster'])):
        #print(team_rosters_2017_2018['teams'][i]['roster']['roster'][j]['person'])
        player = [team_rosters_2017_2018['teams'][i]['roster']['roster'][j]['person']['id'], 
                  team_rosters_2017_2018['teams'][i]['roster']['roster'][j]['person']['fullName'],
                 team_rosters_2017_2018['teams'][i]['name']]
        if (team_rosters_2017_2018['teams'][i]['roster']['roster'][j]['position']['code'] != 'G'):
            players_2017_2018.append(player)
                                                                    
                                                                    

In [ ]:
for i in range(0, len(players_2017_2018)):
    print(players_2017_2018[i])

Get the stats for each player as an array of [timeOnIce, assists, goals, pim, shots, games, hits, powerPlayGoals, powerPlayPoints, powerPlayTimeOnIce, evenTimeOnIce, penaltyMinutes, faceOffPct, shotPct, gameWinningGoals, overTimeGoals, shortHandedGoals, shortHandedPoints, shortHandedTimeOnIce, blocked, plusMinus, points, shifts, timeOnIcePerGame, evenTimeOnIcePerGame, shortHandedTimeOnIcePerGame, powerPlayTimeOnIcePerGame]

In [ ]:
players_2017_2018_stats = []
for i in range(0, len(players_2017_2018)): 
    stats = requests.get('https://statsapi.web.nhl.com/api/v1/people/' 
                       + str(players_2017_2018[i][0]) 
                       + '/stats?stats=statsSingleSeason&season=20172018').json()
    stats = stats['stats'][0]['splits'][0]['stat']
    stats_array = []
    stats_array.append(stats['timeOnIce'])
    stats_array.append(stats['assists'])
    stats_array.append(stats['goals'])
    stats_array.append(stats['pim'])
    stats_array.append(stats['shots'])
    stats_array.append(stats['games'])
    stats_array.append(stats['hits'])
    stats_array.append(stats['powerPlayGoals'])
    stats_array.append(stats['powerPlayPoints'])
    stats_array.append(stats['powerPlayTimeOnIce'])
    stats_array.append(stats['evenTimeOnIce'])
    stats_array.append(stats['penaltyMinutes'])
    stats_array.append(stats['faceOffPct'])
    stats_array.append(stats['shotPct'])
    stats_array.append(stats['gameWinningGoals'])
    stats_array.append(stats['overTimeGoals'])
    stats_array.append(stats['shortHandedGoals'])
    stats_array.append(stats['shortHandedPoints'])
    stats_array.append(stats['shortHandedTimeOnIce'])
    stats_array.append(stats['blocked'])
    stats_array.append(stats['plusMinus'])
    stats_array.append(stats['points'])
    stats_array.append(stats['shifts'])
    stats_array.append(stats['timeOnIcePerGame'])
    stats_array.append(stats['evenTimeOnIcePerGame'])
    stats_array.append(stats['shortHandedTimeOnIcePerGame'])
    stats_array.append(stats['powerPlayTimeOnIcePerGame'])
    players_2017_2018_stats.append(stats_array)

print(len(players_2017_2018_stats))
print(len(players_2017_2018_stats[0]))
    

Append the stats for each player to their id, name, and team

In [ ]:
skaters_2017_2018 = []
for i in range(0, len(players_2017_2018)):
    skaters_2017_2018.append(players_2017_2018[i] + players_2017_2018_stats[i])


In [ ]:
print(skaters_2017_2018)

Save the skaters array as csv data for anaylsis 

In [ ]:
np.savetxt('data/skaters_2017_2018.csv', skaters_2017_2018, fmt='%s', delimiter=',')

In [ ]:
def get_csv(y1, y2):
    team_rosters = requests.get('https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster&season=' + y1 + y2)
    #print('https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster&season=' + y1 + y2)
    team_rosters = team_rosters.json()
    players= []
    for i in range(0, len(team_rosters['teams'])):
        #print(team_rosters_2017_2018['teams'][i]['name'])
        for j in range(0, len(team_rosters['teams'][i]['roster']['roster'])):
            #print(team_rosters_2017_2018['teams'][i]['roster']['roster'][j]['person'])
            player = [team_rosters['teams'][i]['roster']['roster'][j]['person']['id'], 
                      team_rosters['teams'][i]['roster']['roster'][j]['person']['fullName'],
                     team_rosters['teams'][i]['name']]
            if (team_rosters['teams'][i]['roster']['roster'][j]['position']['code'] != 'G'):
                players.append(player)
    players_stats = []
    labels = requests.get('https://statsapi.web.nhl.com/api/v1/people/' 
                           + str(players[i][0]) 
                           + '/stats?stats=statsSingleSeason&season=' + y1 + y2).json()
    labels = labels['stats'][0]['splits'][0]['stat']
    header = ['id', 'fullName', 'teamName']
    for label in labels:
        header.append(label)
    #print(labels)
    #print(header)
    for i in range(0, len(players)): 
        #print(str(players[i][0]))
        stats = requests.get('https://statsapi.web.nhl.com/api/v1/people/' 
                           + str(players[i][0]) 
                           + '/stats?stats=statsSingleSeason&season=' + y1 + y2).json()
        if(stats['stats'][0]['splits'] == []):
            players_stats.append([0] * len(labels))
            continue
        stats = stats['stats'][0]['splits'][0]['stat']
        
        stats_array = []
        for label in labels:
            if label in stats:
                stats_array.append(stats[label])
            else:
                stats_array.append(0)
        players_stats.append(stats_array)
        
    skaters = []
    skaters.append(header)
    for i in range(0, len(players)):
        skaters.append(players[i] + players_stats[i])
    return skaters

In [ ]:
skaters = get_csv('2016', '2017')
np.savetxt('data/skaters_2016_2017.csv', skaters, fmt='%s', delimiter=',')

In [ ]:
skaters = get_csv('2015', '2016')
np.savetxt('data/skaters_2015_2016.csv', skaters, fmt='%s', delimiter=',')

In [ ]:
skaters = get_csv('2014', '2015')
np.savetxt('data/skaters_2014_2015.csv', skaters, fmt='%s', delimiter=',')

In [ ]:
skaters = get_csv('1970', '1971')
np.savetxt('data/skaters_1970_1971.csv', skaters, fmt='%s', delimiter=',')

In [ ]:
stats = requests.get('https://statsapi.web.nhl.com/api/v1/people/' 
                       + str(8445135) 
                       + '/stats?stats=statsSingleSeason&season=19171918').json()

In [ ]:
stats

In [ ]:
stats = stats = stats['stats'][0]['splits'][0]['stat']

In [ ]:
player_stats = []
for x in stats:
    player_stats.append(x)

In [ ]:
player_stats

In [ ]:
def get_data(start, end):
    for i in range(start, end):
        if(i == 2004):
            continue
        print("Getting data for " + str(i) + "-" + str(i+1) + " season.")
        skaters = get_csv(str(i), str(i+1))
        np.savetxt('data/skaters_' + str(i) + '_' + str(i+1) + '.csv', skaters, fmt='%s', delimiter=',')

In [ ]:
get_data(2005,2018)

Get Team data

In [ ]:
teams = requests.get('https://statsapi.web.nhl.com/api/v1/teams?season=20172018')
teams = teams.json()

In [ ]:
team_id_name = []
for i in range(0, len(teams['teams'])):
    team_arr = [teams['teams'][i]['id'], teams['teams'][i]['name']]
    team_id_name.append(team_arr)
#print(team_id_name)

labels = requests.get('https://statsapi.web.nhl.com/api/v1/teams/' 
                           + str(team_id_name[0][0])
                           + '/stats?stats=statsSingleSeason&season=19901991').json()
labels = labels['stats'][0]['splits'][0]['stat']
header = ['id', 'teamName']
for label in labels:
    header.append(label)
#print(header)

team_stats = []
for i in range(0, len(team_id_name)):
    stats = requests.get('https://statsapi.web.nhl.com/api/v1/teams/' 
                         + str(team_id_name[i][0]) 
                         + '/stats?stats=statsSingleSeason&season=19901991').json()
    if(stats['stats'][0]['splits'] == []):
        team_stats.append([0] * len(labels))
        continue
    stats = stats['stats'][0]['splits'][0]['stat']
    stats_array = []
    for label in labels:
        if label in stats:
            stats_array.append(stats[label])
        else:
            stats_array.append(0)
    team_stats.append(stats_array)

    
teams_stats_final = []
teams_stats_final.append(header)
for i in range(0, len(team_id_name)):
    teams_stats_final.append(team_id_name[i] + team_stats[i])    

np.savetxt('team_data/teams_19901991.csv', teams_stats_final, fmt='%s', delimiter=',')

In [3]:
def get_csv_team(y1, y2):
    teams = requests.get('https://statsapi.web.nhl.com/api/v1/teams?season=' + str(y1) + str(y2))
    teams = teams.json()
    team_id_name = []
    for i in range(0, len(teams['teams'])):
        team_arr = [teams['teams'][i]['id'], teams['teams'][i]['name']]
        team_id_name.append(team_arr)

    labels = requests.get('https://statsapi.web.nhl.com/api/v1/teams/' 
                           + str(team_id_name[0][0])
                           + '/stats?stats=statsSingleSeason&season=' + str(y1) + str(y2)).json()
    labels = labels['stats'][0]['splits'][0]['stat']
    header = ['id', 'teamName']
    for label in labels:
        header.append(label)

    team_stats = []
    for i in range(0, len(team_id_name)):
        stats = requests.get('https://statsapi.web.nhl.com/api/v1/teams/' 
                             + str(team_id_name[i][0]) 
                             + '/stats?stats=statsSingleSeason&season=' + str(y1) + str(y2)).json()
        if(stats['stats'][0]['splits'] == []):
            team_stats.append([0] * len(labels))
            continue
        stats = stats['stats'][0]['splits'][0]['stat']
        stats_array = []
        for label in labels:
            if label in stats:
                stats_array.append(stats[label])
            else:
                stats_array.append(0)
        team_stats.append(stats_array)

    
    teams_stats_final = []
    teams_stats_final.append(header)
    for i in range(0, len(team_id_name)):
        teams_stats_final.append(team_id_name[i] + team_stats[i]) 
    return teams_stats_final

In [9]:
def get_team_data(start, end):
    for i in range(start, end):
        if(i == 2004):
            continue
        print("Getting data for " + str(i) + "-" + str(i+1) + " season.")
        data = get_csv_team(str(i), str(i+1))
        np.savetxt('team_data/teams_' + str(i) + '_' + str(i+1) + '.csv', data, fmt='%s', delimiter=',')

In [7]:
get_team_data(1917,2018)

Getting data for 1917-1918 season.
Getting data for 1918-1919 season.
Getting data for 1919-1920 season.
Getting data for 1920-1921 season.
Getting data for 1921-1922 season.
Getting data for 1922-1923 season.
Getting data for 1923-1924 season.
Getting data for 1924-1925 season.
Getting data for 1925-1926 season.
Getting data for 1926-1927 season.
Getting data for 1927-1928 season.
Getting data for 1928-1929 season.
Getting data for 1929-1930 season.
Getting data for 1930-1931 season.
Getting data for 1931-1932 season.
Getting data for 1932-1933 season.
Getting data for 1933-1934 season.
Getting data for 1934-1935 season.
Getting data for 1935-1936 season.
Getting data for 1936-1937 season.
Getting data for 1937-1938 season.
Getting data for 1938-1939 season.
Getting data for 1939-1940 season.
Getting data for 1940-1941 season.
Getting data for 1941-1942 season.
Getting data for 1942-1943 season.
Getting data for 1943-1944 season.
Getting data for 1944-1945 season.
Getting data for 194

IndexError: list index out of range

Getting data for 2003-2004 season.
